# **8. Bevezetés a gépi látás mélytanulásába**

Ez a fejezet ezekkel foglalkozik:
* A konvolúciós neurális hálózatok (convnet - CNN) megértése
* Adatbővítés használata a túlillesztés csökkentésére
* Előképzett convnet használata a jellemzők kinyeréséhez
* Előképzett convnet finomhangolása

A gépi látás (CV = Computer Vision) a mély tanulás legkorábbi és legnagyobb sikertörténete. Minden nap kapcsolatba kerülünk a mély-látó modellekkel – a Google Fotókon, a Google képkeresőn, a YouTube-on, a kameraalkalmazások videószűrőin, az OCR-szoftveren és még sok máson keresztül. Ezek a modellek az önvezetés, a robotika, a mesterséges intelligencia által támogatott orvosi diagnosztika, az autonóm kiskereskedelmi pénztárrendszerek és még az önálló mezőgazdálkodás terén folyó kutatások középpontjában is ott állnak.

A gépi látás az a problémakör, amely a mély tanulás kezdeti térnyeréséhez vezetett 2011 és 2015 között. A *konvolúciós neurális hálózatoknak* (CNN) nevezett mélytanulási modell ekkortájt kezdett figyelemreméltóan jó eredményeket elérni a képosztályozási versenyeken, először Dan Ciresan nyert két rétegversenyen (az ICDAR 2011 kínai karakterfelismerő verseny és az IJCNN 2011 német közlekedési táblák felismerési versenye), majd különösen 2012 őszén, amikor Hinton csoportja megnyerte a nagy horderejű ImageNet nagyszabású vizuális felismerési kihívását. Gyorsan sok ígéretesebb eredmény kezdett felbukkanni más gépi látási feladatokban is.

Érdekes módon ezek a korai sikerek nem voltak elégségesek ahhoz, hogy a mély tanulást akkoriban általánossá tegyék – ez néhány évbe telt. A gépi látást kutató közösség már sok évet eltöltött azzal, hogy neurális hálózatokon kívüli más módszerekbe fektessen be, és nem volt egészen készen arra, hogy lemondjon róluk, csak azért, mert egy új gyerek jött a háztömbbe. 2013-ban és 2014-ben sok vezető gépi látást kutató még mindig heves szkepticizmussal viseltetett a mély tanulással kapcsolatban. Csak 2016-ban vált végül dominánssá. Emlékszem, 2014 februárjában buzdítottam egy volt professzoromat, hogy forduljon a mély tanulás felé. – "Ez a következő nagy dolog!" - ezt mondtam. „Nos, talán ez csak egy hóbort” – válaszolta. 2016-ra az egész laborja mélytanulást végzett. Nem lehet megállítani egy ötletet, amelynek eljött az ideje.

Ez a fejezet bemutatja a konvolúciós neurális hálózatokat, más néven *convnet*-eket, a mély tanulási modell azon típusát, amelyet ma már szinte általánosan használnak a gépi látási alkalmazásokban. Megtanulhatja a convnet alkalmazását képbesorolási problémákra – különösen azokra, amelyekkel kis betanítási adatkészlet jár, és amelyek a leggyakoribb felhasználási esetek, ha ön nem egy nagy technológiai vállalat.

## 8.1 Bevezetés a convnetbe

Hamarosan belemerülünk abba az elméletbe, hogy mik azok a convnet-ek, és miért olyan sikeresek a gépi látási feladatokban. Először azonban vessünk egy gyakorlati pillantást egy egyszerű convnet példára, amely az MNIST számjegyeket osztályozza. Ezt a feladatot a 2. fejezetben egy sűrűn összekötött hálózat segítségével végeztük el (a teszt pontossága akkor 97,8% volt). Annak ellenére, hogy a convnet alapszintű lesz, a pontossága teljesen legyőzi a 2. fejezetből származó, sűrűn összekötött modellünket.

A következő lista bemutatja, hogyan néz ki egy alapvető convnet. Ez egy halom `Conv2D` és `MaxPooling2D` réteg. Egy percen belül látni fogja, hogy pontosan mit csinálnak. A modellt az előző fejezetben bemutatott Funkcionális API segítségével készítjük el.

**8.1 lista: Egy kis convnet példányosítása**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

Fontos, hogy a convnet az alakzat bemeneti tenzorait veszi fel `(image_height, image_width, image_channels)`, a kötegdimenzió nélkül. Ebben az esetben a convnet-et úgy állítjuk be, hogy a `(28, 28, 1)` méretű bemeneteket dolgozza fel, ami az MNIST képek formátuma.

Mutassuk meg a convnetünk architektúráját.

**8.2 lista: A modell összegzésének megjelenítése**

```
>>> model.summary()
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
=================================================================
input_1 (InputLayer)         [(None, 28, 28, 1)]       0
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0
_________________________________________________________________
dense (Dense)                (None, 10)                11530
=================================================================
Total params: 104,202
Trainable params: 104,202
Non-trainable params: 0
_________________________________________________________________
```

Látható, hogy minden `Conv2D` és `MaxPooling2D` réteg kimenete egy `(height, width, channels)` alakú harmadrendű tenzor. A szélesség és magasság méretek általában csökkennek, ahogy mélyebbre megyünk a modellben. A csatornák számát a `Conv2D` rétegeknek átadott első argumentum (32, 64 vagy 128) szabályozza.

Az utolsó `Conv2D` réteg után a (3, 3, 128) alakú kimenetet kapjuk – egy 128 csatornából álló 3 × 3-as jellemzőtérképet. A következő lépés ennek a kimenetnek a betáplálása egy sűrűn összekapcsolt osztályozóba, mint amilyeneket már ismerünk: egy halom `Dense` réteget. Ezek az osztályozók vektorokat dolgoznak fel, amelyek 1D-sek, míg az aktuális kimenet egy harmadrendű tenzor. A rés áthidalása érdekében a 3D kimeneteket 1D-re simítjuk egy `Flatten` réteggel, mielőtt hozzáadnánk a `Dense` rétegeket.

Végül 10 utas osztályozást végzünk, így az utolsó rétegünk 10 kimenettel és softmax aktiválással rendelkezik.

Most pedig tanítsuk be a convnetet az MNIST számjegyekre. A 2. fejezetben található MNIST-példa kódjának nagy részét újra felhasználjuk. Mivel 10-utas osztályozást végzünk softmax kimenettel, a kategorikus keresztentrópia veszteséget fogjuk alkalmazni, és mivel a címkéink egész számok, a ritka változatát fogjuk használni, a `sparse_categorical_crossentropy`-t.

**8.3 lista: A convnet betanítása MNIST képeken**

In [ ]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Értékeljük a modellt a tesztadatok alapján.

**8.4 lista: A convnet kiértékelése**

In [ ]:
>>> test_loss, test_acc = model.evaluate(test_images, test_labels)
>>> print(f"Test accuracy: {test_acc:.3f}")
Test accuracy: 0.991

Míg a 2. fejezetben található sűrűn összekapcsolt modell tesztpontossága 97,8%, addig az alap convnet tesztpontossága 99,1%: a hibaarányt körülbelül 60%-kal csökkentettük (relatív). Nem rossz!

De miért működik olyan jól ez az egyszerű convnet egy sűrűn összekapcsolt modellhez képest? Ennek megválaszolásához nézzük meg, mit csinál a `Conv2D` és a `MaxPooling2D` réteg.

### 8.1.1 A konvolúció művelet

Az alapvető különbség a sűrűn összefüggő réteg és a konvolúciós réteg között a következő: a `Dense` rétegek globális mintákat tanulnak meg bemeneti jellemzőterükben (például egy MNIST számjegy esetén az összes képpontot magában foglaló mintákat), míg a konvolúciós rétegek helyi mintákat tanulnak meg – ebben az esetben a bemenetek kis 2D-s ablakaiban található képeket, mintákat (lásd 8.1. ábra). Az előző példában ezek az ablakok mind 3 × 3 méretűek voltak.

![](figs/f8.1_.jpg)

**8.1. ábra:** A képek helyi mintákra bonthatók, például élekre, textúrákra stb.

Ez a fő jellemző két érdekes tulajdonságot biztosít a convneteknek:
* *Az általuk tanult minták eltolás-invariánsak.* Miután megtanult egy bizonyos mintát a kép jobb alsó sarkában, a convnet bárhol képes felismerni: például a bal felső sarokban. Egy sűrűn összekapcsolt modellnek újra meg kell tanulnia a mintát, ha új helyen jelenik meg. Ez teszi a convneteket hatékonnyá képek feldolgozásakor (mivel _a vizuális világ alapvetően áthelyezés-invariáns_): kevesebb betanítási mintára van szükségük ahhoz, hogy megtanulják az általánosító erővel bíró reprezentációkat.
* *Képesek megtanulni a minták térbeli hierarchiáját.* Az első konvolúciós réteg kis helyi mintákat, például éleket, a második konvolúciós réteg az első réteg jellemzőiből készült nagyobb mintákat, és így tovább (lásd a 8.2. ábrát). Ez teszi lehetővé a convnet számára, hogy hatékonyan tanuljon meg egyre összetettebb és absztraktabb vizuális fogalmakat, mivel _a vizuális világ alapvetően térben hierarchikus_.

![](figs/f8.2_.jpg)

**8.2. ábra:** A vizuális világot a vizuális modulok térbeli hierarchiája alkotja:
az elemi vonalak vagy textúrák egyszerű tárgyakká, például szemekké vagy fülekké egyesülnek, amelyekből olyan magas szintű fogalmak születnek, mint a „macska”.

A konvolúciók harmadrendű tenzorokon, úgynevezett _jellemzőtérképeken_ működnek, két térbeli tengellyel (_magasság_ és _szélesség_), valamint egy _mélységi_ tengellyel (más néven _csatornatengely_). RGB kép esetén a mélységi tengely mérete 3, mivel a képnek három színcsatornája van: piros, zöld és kék. Fekete-fehér képnél, mint az MNIST számjegyek, a mélység 1 (szürke szintek). A konvolúciós művelet foltokat von ki a bemenet jellemzőtérképéből, és ugyanazt a transzformációt alkalmazza erre az összes foltra, így egy _kimeneti jellemzőtérképet_ hoz létre. Ez a kimeneti jellemzőtérkép még mindig harmadrendű tenzor: van szélessége és magassága. Mélysége tetszőleges lehet, mert a kimeneti mélység a réteg paramétere, és a különböző csatornák ezen a mélységtengelyen már nem jelölik az adott színt, mint az RGB bemenetnél; inkább _szűrőket_ jelentenek. A szűrők a bemeneti adatok meghatározott szempontjait kódolják: magas szinten egyetlen szűrő kódolni képes például az „arc jelenléte a bemenetben” fogalmat.

Az MNIST példában az első konvolúciós réteg egy (28, 28, 1) méretű jellemzőtérképet vesz át, és egy (26, 26, 32) méretű jellemzőtérképet ad ki: 32 szűrőt számít ki a bemenetére. Mind a 32 kimeneti csatorna egy 26 × 26-os értékrácsot tartalmaz, amely a szűrő _választérképe_ a bemenetre, jelezve a szűrőmintázat válaszát a bemenet különböző helyein (lásd a 8.3. ábrát).

![](figs/f8.3_.jpg)

**8.3. ábra:** A választérkép fogalma: egy minta 2D-s térképe a bemenet különböző helyein

Ezt jelenti a _jellemzőtérkép_ kifejezés: a mélységi tengely minden dimenziója egy jellemző (vagy egy szűrő), és az `output[:, :, n]` másodrendű tenzor a szűrő bemenetre adott válaszának 2D-s térbeli _térképe_.

A konvolúciókat két fő paraméter határozza meg:
* _A bemenetekből kinyert foltok mérete_ – ezek általában 3 × 3 vagy 5 × 5 méretűek. A példában 3 × 3 méretűek voltak, ami gyakori választás.
* _A kimeneti jellemzőtérkép mélysége_ – Ez a konvolúció által kiszámított szűrők száma. A példa 32-es mélységgel kezdődött és 64-es mélységgel ért véget.

A Keras `Conv2D` rétegekben ezek a paraméterek a rétegnek átadott első argumentumok: `Conv2D(output_depth, (window_height, window_width))`.

A konvolúció úgy működik, hogy ezeket a 3 × 3 vagy 5 × 5 méretű ablakokat _elcsúsztatja_ a 3D beviteli jellemzőtérképen, megáll minden lehetséges helyen, és kivonja a környező elemek 3D foltját (`(window_height, window_width, input_depth)` alakú). Minden egyes ilyen 3D foltot ezután egy `(output_depth,)` alkú 1D vektorrá alakít át, ami egy megtanult súlymátrixszal való tenzorszorzaton keresztül történik, amelyet _konvolúciós kernelnek_ neveznek – ugyanazt a kernelt használjuk fel újra minden folton. Mindezek a vektorok (foltonként egy) azután térben újból összeállnak egy `(height, width, output_
depth)` alakú 3D kimeneti térképbe. A kimeneti jellemzőtérkép minden térbeli helye megegyezik a bemeneti jellemzőtérkép azonos helyével (például a kimenet jobb alsó sarka a bemenet jobb alsó sarkáról tartalmaz információkat). Például 3 × 3 ablaknál az `output[i, j, :]` vektor az `input[i-1:i+1, j-1:j+1, :]` 3D foltból származik. A teljes folyamatot a 8.4. ábra részletezi.

![](figs/f8.4_.jpg)

**8.4. ábra:** A konvolúció működése

Vegye figyelembe, hogy a kimeneti szélesség és magasság két okból is eltérhet a bemeneti szélességtől és magasságtól:
* Szegélyhatások, amelyek ellensúlyozhatók a bemeneti jellemzőtérkép kitömésével
* A _lépések_ használata, amit egy pillanat múlva definiálok

Nézzük meg mélyebben ezeket a fogalmakat.

**A SZEGÉLYHATÁSOK ÉS A KITÖMÉS MEGÉRTÉSE**

Vegyünk egy 5 × 5 méretű jellemzőtérképet (összesen 25 kocka). Csak 9 kocka van, amelyek köré egy 3 × 3-as ablakot középre tudunk helyezni, és ez egy 3 × 3-as rácsot alkot (lásd a 8.5. ábrát). Ezért a kimeneti jellemzőtérkép 3 × 3 lesz. Kicsit zsugorodik: ebben az esetben pontosan két kockával mindegyik dimenzió irányában. Ezt a szegélyeffektust láthatja működés közben a korábbi példában: 28 × 28 bemenettel kezdi, amely az első konvolúciós réteg után 26 × 26 lesz.

![](figs/f8.5_.jpg)

**8.5. ábra:** 3 × 3-as foltok érvényes helyei egy 5 × 5-ös bemeneti jellemzőtérképen

Ha olyan kimeneti jellemzőtérképet szeretnénk kapni, amelynek térbeli méretei megegyeznek a bemenettel, használhatunk kitömést. A kitömés abból áll, hogy megfelelő számú sort és oszlopot adunk hozzá a beviteli jellemzőtérkép mindkét oldalához, úgy hogy minden beviteli csempe köré központosan konvolúciós ablakot lehessen illeszteni. Egy 3 × 3-as ablakhoz egy oszlopot adunk hozzá a jobb oldalon, egy oszlopot a bal oldalon, egy sort a tetején és egy sort alul. Egy 5 × 5-ös ablakhoz két sort kell hozzáadni (lásd a 8.6. ábrát).

![](figs/f8.6_.jpg)

**8.6 ábra:** 5 × 5-ös bemenet kitömése úgy, hogy 25 darab 3 × 3-as foltot tudjunk kiválasztani

A `Conv2D` rétegekben a kitömés a `padding` argumentummal konfigurálható, amely két értéket vesz fel: `"valid"`, ami azt jelenti, hogy nincs kitöltés (csak az érvényes ablakhelyek kerülnek felhasználásra), és `"same"`, ami azt jelenti, hogy "kitöltés oly módon, hogy ugyanolyan szélességű és magasságú kimenettel rendelkezzen, mint a bemenet." A padding argumentum alapértelmezés szerint `"valid"`.

**A KONVOLUCIÓS LÉPÉSEK MEGÉRTÉSE**

A másik tényező, amely befolyásolhatja a kimenet méretét, a _lépések_ fogalma. A konvolúció eddigi leírása abból indult ki, hogy a konvolúciós ablakok középső kockái mind határosak egymással. De a két egymást követő ablak közötti távolság a konvolúció egyik paramétere, amelyet `strides`-nek neveznek, és amely alapértelmezés szerint 1. Lehetnek _lépcsőzetes konvolúciók_ is: olyan konvolúciók, amelyek lépésszáma nagyobb, mint 1. A 8.7. ábrán láthatók az olyan foltok, amelyeket egy 3 × 3 konvolúció választ ki 2 lépéssel 5 × 5 bemeneten (kitömés nélkül).

![](figs/f8.7_.jpg)

**8.7. ábra:** 3 × 3 konvolúciós foltok 2 × 2 lépéssel

A 2 lépés használata azt jelenti, hogy a jellemzőtérkép szélességét és magasságát 2-szeres mintavételezéssel csökkentjük (a szegélyeffektusok által kiváltott változásokon kívül). A lépcsőzetes konvolúciót ritkán alkalmazzák az osztályozási modellekben, de bizonyos típusú modelleknél jól jönnek, amint azt a következő fejezetben látni fogjuk.
Az osztályozási modellekben a lépések helyett inkább a _max-pooling_ műveletet használjuk a jellemzőtérképek mintavételezésére, amit az első convnet példánkban mér láthattunk. Nézzük meg alaposabban.

### 8.1.2 A max-pooling művelet

A convnet példában észrevehette, hogy a jellemzőtérképek mérete felére csökken minden `MaxPooling2D` réteg után. Például az első `MaxPooling2D` rétegek előtt a jellemzőtérkép 26 × 26 méretű, de a max-pooling művelet felezi azt 13 × 13-ra. Ez a max-pooling szerepe: a jellemzőtérképek agresszív alulmintavételezése, hasonlóan a lépcsős konvolúcióhoz.

A max pooling abból áll, hogy kiválasztja az ablakokat a bemeneti jellemzőtérképekből, és kiadja az egyes csatornák maximális értékét. Elvileg hasonló a konvolúcióhoz, azzal a különbséggel, hogy ahelyett, hogy a helyi foltokat egy tanult lineáris transzformációval (a konvolúciós kernel) transzformálnák, egy keménykódolt `max` tenzorművelettel alakítják át őket. Nagy különbség a konvolúcióhoz képest, hogy a max pooling általában 2 × 2 ablakkal és 2 lépéssel történik, hogy a jellemzőtérképeket felére csökkentsék. Másrészt a konvolúciót általában 3 × 3 ablakkal végzik, és lépés nélkül (a lépés 1). {210.o:}

Miért kell így alulmintavételezni a jellemzőtérképeket? Miért ne távolítsa el a max-pooling rétegeket, és tartsa fenn inkább a meglehetősen nagy tereptérképeket mindvégig? Nézzük ezt a lehetőséget. A modellünk így nézne ki, mint az alábbi listán.

**8.5. lista: Egy helytelenül strukturált convnet, amelyből hiányoznak a max-pooling rétegei**

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)